In [1]:
import pandas as pd 
import numpy as np      
    

county_vax_df = pd.read_csv("covid-death-data.csv")
#Link to this Data set  
#https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh 
# Make sure we also turn in this csv file with the submission 

vax_df = pd.read_json('https://data.cdc.gov/resource/d6p8-wqjm.json') 
#Link to this Data Set:  
#https://data.cdc.gov/Public-Health-Surveillance/Rates-of-COVID-19-Cases-or-Deaths-by-Age-Group-and/d6p8-wqjm

## Data Sets for the Personalized Graphs

In [2]:
vax_df = vax_df.drop(columns=['crude_booster_ir', 'crude_primary_series_only_ir','crude_unvax_ir', 
                              'crude_booster_irr', 'crude_irr', 'continuity_correction','age_adj_booster_ir', 
                              'age_adj_vax_ir', 'age_adj_unvax_ir','age_adj_booster_irr', 'age_adj_irr','month'])   
 
vax_df['mmwr_week'] = vax_df['mmwr_week']*10 + 0
vax_df['date'] = pd.to_datetime(vax_df['mmwr_week'], format='%Y%W%w')
print(vax_df.shape)
vax_df.head()       

(825, 11)


,outcome,mmwr_week,age_group,vaccine_product,boosted_with_outcome,boosted_population,primary_series_only_with_outcome,primary_series_only_population,unvaccinated_with_outcome,unvaccinated_population,date
0,death,2022300,5-11,all_types,0,545135,0,4894771,1,12774055.00,2022-07-31
1,death,2022300,12-17,all_types,0,2786599,0,7069903,0,6035472.00,2022-07-31
2,death,2022300,18-49,all_types,11,25207882,12,37239463,39,23658679.75,2022-07-31
3,case,2021380,18-49,all_types,119,307241,52609,52473497,170089,34939594.00,2021-09-26
4,case,2021380,50-64,all_types,147,376834,28091,30068369,46391,9714056.00,2021-09-26


In [3]:
vax_df = vax_df.drop(columns=['mmwr_week'],axis=1)

In [4]:
vax_df = vax_df[(vax_df['age_group']!= '5-11')&(vax_df['age_group']!='12-17')]

In [5]:
case_df = vax_df[vax_df['outcome']=='case'] 
death_df = vax_df[vax_df['outcome']=='death']

In [6]:
df1 = case_df.set_index('date').groupby([pd.Grouper(freq='W'),'age_group','vaccine_product']).sum() 
df1.head()

boosted_with_outcome  \
date       age_group vaccine_product                         
2021-09-26 18-49     all_types                         119   
           50-64     all_types                         147   
           65+       all_types                         280   
           all_ages  Janssen                             7   
                     Moderna                           202   

                                      boosted_population  \
date       age_group vaccine_product                       
2021-09-26 18-49     all_types                    307241   
           50-64     all_types                    376834   
           65+       all_types                    878779   
           all_ages  Janssen                       28692   
                     Moderna                      685545   

                                      primary_series_only_with_outcome  \
date       age_group vaccine_product                                     
2021-09-26 18-49     all_types                                   52609   
           50-64     all_types                                   28091   
           65+       all_types                                   27243   
           all_ages  Janssen                                     11003   
                     Moderna                                     35197   

                                      primary_series_only_population  \
date       age_group vaccine_product                                   
2021-09-26 18-49     all_types                              52473497   
           50-64     all_types                              30068369   
           65+       all_types                              29106029   
           all_ages  Janssen                                 9567298   
                     Moderna                                44214235   

                                      unvaccinated_with_outcome  \
date       age_group vaccine_product                              
2021-09-26 18-49     all_types                           170089   
           50-64     all_types                            46391   
           65+       all_types                            24190   
           all_ages  Janssen                             240670   
                     Moderna                             240670   

                                      unvaccinated_population  
date       age_group vaccine_product                           
2021-09-26 18-49     all_types                     34939594.0  
           50-64     all_types                      9714056.0  
           65+       all_types                      4820712.8  
           all_ages  Janssen                       49474362.8  
                     Moderna                       49474362.8

In [7]:
df2 = death_df.set_index('date').groupby([pd.Grouper(freq='W'),'age_group','vaccine_product']).sum() 
df2.head()

boosted_with_outcome  \
date       age_group vaccine_product                         
2021-09-26 18-49     all_types                           0   
           50-64     all_types                           2   
           65+       all_types                          18   
           all_ages  Janssen                             0   
                     Moderna                            11   

                                      boosted_population  \
date       age_group vaccine_product                       
2021-09-26 18-49     all_types                    297687   
           50-64     all_types                    360891   
           65+       all_types                    837953   
           all_ages  Janssen                       29242   
                     Moderna                      656886   

                                      primary_series_only_with_outcome  \
date       age_group vaccine_product                                     
2021-09-26 18-49     all_types                                      46   
           50-64     all_types                                     150   
           65+       all_types                                    1082   
           all_ages  Janssen                                       113   
                     Moderna                                       473   

                                      primary_series_only_population  \
date       age_group vaccine_product                                   
2021-09-26 18-49     all_types                              49768966   
           50-64     all_types                              28284539   
           65+       all_types                              27267237   
           all_ages  Janssen                                 9011732   
                     Moderna                                41701814   

                                      unvaccinated_with_outcome  \
date       age_group vaccine_product                              
2021-09-26 18-49     all_types                              696   
           50-64     all_types                             1421   
           65+       all_types                             2388   
           all_ages  Janssen                               4505   
                     Moderna                               4505   

                                      unvaccinated_population  
date       age_group vaccine_product                           
2021-09-26 18-49     all_types                     33554194.0  
           50-64     all_types                      9260611.0  
           65+       all_types                      4766759.8  
           all_ages  Janssen                       47581564.8  
                     Moderna                       47581564.8

In [8]:
# Reset index to create separate columns for `date`, `age_group`, `outcome`, and `vaccine_product`
df1 = df1.reset_index()

# Melt the dataframe to transform it into a long format
melted_case_data = pd.melt(df1, id_vars=['date', 'age_group','vaccine_product'], value_vars=['boosted_with_outcome', 'primary_series_only_with_outcome', 'unvaccinated_with_outcome'], var_name='Vaccination Status')
melted_case_data.head()  

,date,age_group,vaccine_product,Vaccination Status,value
0,2021-09-26,18-49,all_types,boosted_with_outcome,119
1,2021-09-26,50-64,all_types,boosted_with_outcome,147
2,2021-09-26,65+,all_types,boosted_with_outcome,280
3,2021-09-26,all_ages,Janssen,boosted_with_outcome,7
4,2021-09-26,all_ages,Moderna,boosted_with_outcome,202


In [9]:
# Reset index to create separate columns for `date`, `age_group`, `outcome`, and `vaccine_product`
df2 = df2.reset_index()

# Melt the dataframe to transform it into a long format
melted_death_data = pd.melt(df2, id_vars=['date', 'age_group','vaccine_product'], value_vars=['boosted_with_outcome', 'primary_series_only_with_outcome', 'unvaccinated_with_outcome'], var_name='Vaccination Status') 
melted_death_data.head()  

,date,age_group,vaccine_product,Vaccination Status,value
0,2021-09-26,18-49,all_types,boosted_with_outcome,0
1,2021-09-26,50-64,all_types,boosted_with_outcome,2
2,2021-09-26,65+,all_types,boosted_with_outcome,18
3,2021-09-26,all_ages,Janssen,boosted_with_outcome,0
4,2021-09-26,all_ages,Moderna,boosted_with_outcome,11


## Data set for Universal Graph (the map) 

In [10]:
county_vax_df.head()

,county,county_fips,state,county_population,health_service_area_number,health_service_area,health_service_area_population,covid_inpatient_bed_utilization,covid_hospital_admissions_per_100k,covid_deaths_per_500k,covid-19_community_level,date_updated
0,Lincoln County,55069,Wisconsin,"27,593",282,"Marathon (Wausau), WI - Wood, WI","291,401",4.7,13.4,177.58,Medium,8/18/2022
1,Manitowoc County,55071,Wisconsin,"78,981",355,"Sheboygan (Sheboygan), WI - Manitowoc, WI","244,410",3.4,9.8,169.66,Low,8/18/2022
2,Marathon County,55073,Wisconsin,"135,692",282,"Marathon (Wausau), WI - Wood, WI","291,401",4.7,13.4,209.3,High,8/18/2022
3,Monroe County,55081,Wisconsin,"46,253",290,"La Crosse (La Crosse), WI - Monroe, WI","257,027",3.9,15.6,216.2,High,8/18/2022
4,Portage County,55097,Wisconsin,"70,772",400,"Portage, WI","70,772",5.9,7.1,217.6,Medium,8/18/2022


## The DashBoard 

In [11]:
import altair as alt
from vega_datasets import data
import pandas as pd

counties = alt.topo_feature(data.us_10m.url, 'counties')
states = alt.topo_feature(data.us_10m.url, 'states')
source = county_vax_df

deaths = melted_death_data  
cases = melted_case_data
#print(deaths.head())
#print(type(deaths))

age_group = ['18-49', '50-64', '65+', 'all_ages']   

vaccine_product = ['all_types', 'Janssen', 'Moderna', 'Pfizer']

age_dropdown = alt.binding_select(options=age_group, name='Select Age Group')

vaccine_dropdown = alt.binding_select(options = vaccine_product, name='Select Vaccine Product') 

age_selection = alt.selection_single(fields=['age_group'], bind=age_dropdown)

vaccine_selection = alt.selection_single(fields =['vaccine_product'],bind=vaccine_dropdown)

line = alt.Chart(deaths.reset_index()).mark_line().encode(
    x = 'date',
    y = 'value:Q', 
    color = 'Vaccination Status:N',
    tooltip = ['date', 'value'] 
).properties(
    title=['Deaths by Vaccinated Population per Week']  
).add_selection( 
    age_selection 
).add_selection( 
    vaccine_selection 
).transform_filter( 
    age_selection  
).transform_filter( 
    vaccine_selection 
).interactive()

  
                           
line2 = alt.Chart(cases.reset_index()).mark_line().encode(
    x = 'date',
    y = 'value:Q', 
    color = 'Vaccination Status:N',
    tooltip = ['date', 'value'] 
).properties(
    title=['Cases by Vaccinated Population per Week']  
).add_selection( 
    age_selection 
).add_selection( 
    vaccine_selection 
).transform_filter( 
    age_selection  
).transform_filter( 
    vaccine_selection 
).interactive() 
                           



highlight = alt.selection_single(on='mouseover', fields=['id'], empty='none')

plot = alt.Chart(counties).mark_geoshape().encode(
    color=alt.condition(highlight, alt.value('red'), 'covid_deaths_per_500k:Q'),
    tooltip=['covid_deaths_per_500k:Q', 'county:N']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'county_fips', ['covid_deaths_per_500k', 'county'])
).add_selection(highlight).project(
    type='albersUsa'
).properties(
    width=900,
    height=600,
)

outline = alt.Chart(states).mark_geoshape(stroke='blue', fillOpacity=0).project(
    type='albersUsa'
).properties(
    width=700,
    height=400
)

m = alt.layer(plot,outline).properties(title=["Covid Related Deaths per 500k by County"])   


In [12]:
final_plots = m & line & line2

In [13]:
final_plots

alt.VConcatChart(...)

In [ ]:
!pip install altair_saver

In [ ]:
import altair_saver

Save the chart as an HTML file
altair_saver.save(final_plots, 'INFO Visualization Final Project.html')

In [ ]:
from IPython.display import HTML

Define the file path of the HTML file
file_path = './INFO Visualization Final Project.html'

Create an HTML string with a hyperlink to the HTML file
html_string = f'<a href="{file_path}" target="_blank">Open HTML file</a>'

Display the HTML string in the notebook output
HTML(html_string)
